<a href="https://colab.research.google.com/github/rakuri2026/VM0047/blob/main/testing_vcal_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
# Install required packages
!pip install -q streamlit pandas pyngrok

In [45]:
%%writefile app.py
import streamlit as st
import pandas as pd
import geopandas as gpd
import numpy as np
import re
import math
import os
import pydeck as pdk
import json

# ==============================================================
# 1. REFERENCE SPECIES TABLE
# ==============================================================
species_data = {
    'SN': range(1, 26),
    'scientific_name': ['Abies spp', 'Acacia catechu', 'Adina cardifolia', 'Albizia spp', 'Alnus nepalensis',
                       'Anogeissus latifolia', 'Bombax ceiba', 'Cedrela toona', 'Dalbergia sissoo',
                       'Eugenia Jambolana', 'Hymenodictyon excelsum', 'Lagerstroemia parviflora',
                       'Michelia champaca', 'Pinus roxburghii', 'Pinus wallichiana', 'Quercus spp',
                       'Schima wallichii', 'Shorea robusta', 'Terminalia alata', 'Trewia nudiflora',
                       'Tsuga spp', 'Terai spp', 'Hill spp', 'Coniferious', 'Broadleaved'],
    'a': [-2.4453, -2.3256, -2.5626, -2.4284, -2.7761, -2.272, -2.3856, -2.1832, -2.1959, -2.5693,
          -2.585, -2.3411, -2.0152, -2.977, -2.8195, -2.36, -2.7385, -2.4554, -2.4616, -2.4585,
          -2.5293, -2.3993, -2.3204, None, None],
    'b': [1.722, 1.6476, 1.8598, 1.7609, 1.9006, 1.7499, 1.7414, 1.8679, 1.6567, 1.8816,
          1.9437, 1.7246, 1.8555, 1.9235, 1.725, 1.968, 1.8155, 1.9026, 1.8497, 1.8043,
          1.7815, 1.7836, 1.8507, None, None],
    'c': [1.0757, 1.0552, 0.8783, 0.9662, 0.9428, 0.9174, 1.0063, 0.7569, 0.9899, 0.8498,
          0.7902, 0.9702, 0.763, 1.0019, 1.1623, 0.7496, 1.0072, 0.8352, 0.88, 0.922,
          1.0369, 0.9546, 0.8223, None, None],
    'a1': [5.4433, 5.4401, 5.4681, 4.4031, 6.019, 4.9502, 4.5554, 4.9705, 4.358, 5.1749,
           5.5572, 5.3349, 3.3499, 6.2696, 5.7216, 4.8511, 7.4617, 5.2026, 4.5968, 5.3475,
           5.2774, 4.8991, 5.5323, None, None],
    'b1': [-2.6902, -2.491, -2.491, -2.2094, -2.7271, -2.3353, -2.3009, -2.3436, -2.1559, -2.3636,
           -2.496, -2.4428, -2.0161, -2.8252, -2.6788, -2.4494, -3.0676, -2.4788, -2.2305, -2.4774,
           -2.6483, -2.3406, -2.4815, None, None],
    's': [0.436, 0.443, 0.443, 0.443, 0.803, 0.443, 0.443, 0.443, 0.684, 0.443,
          0.443, 0.443, 0.443, 0.189, 0.683, 0.747, 0.52, 0.055, 0.443, 0.443,
          0.443, 0.443, 0.443, 0.436, 0.443],
    'm': [0.372, 0.511, 0.511, 0.511, 1.226, 0.511, 0.511, 0.511, 0.684, 0.511,
          0.511, 0.511, 0.511, 0.256, 0.488, 0.96, 0.186, 0.341, 0.511, 0.511,
          0.511, 0.511, 0.511, 0.372, 0.511],
    'bg': [0.355, 0.71, 0.71, 0.71, 1.51, 0.71, 0.71, 0.71, 0.684, 0.71,
           0.71, 0.71, 0.71, 0.3, 0.41, 1.06, 0.168, 0.357, 0.71, 0.71,
           0.71, 0.71, 0.71, 0.355, 0.71],
    'Local_Name': ['Thingre Salla', 'Khayar', 'Karma', 'Siris', 'Uttis', 'Banjhi', 'Simal', 'Tooni',
                   'Sissoo', 'Jamun', 'Bhudkul', 'Botdhayero', 'Chanp', 'Khote Salla', 'Gobre Salla',
                   'Kharsu', 'Chilaune', 'Sal', 'Saj', 'Gamhari', 'Dhupi Salla', 'Terai Spp',
                   'Hill spp', None, None]
}
df_ref = pd.DataFrame(species_data)

# ==============================================================
# 2. COORDINATE HELPERS
# ==============================================================
COORD_PATTERNS = {
    "lon": re.compile(r"^(lon|longitude|x|east|easting|x.?coord)", re.I),
    "lat": re.compile(r"^(lat|latitude|y|north|northing|y.?coord)", re.I),
}
def guess_coord_cols(df):
    lon = lat = None
    for c in df.columns:
        if COORD_PATTERNS["lon"].search(c): lon = c
        elif COORD_PATTERNS["lat"].search(c): lat = c
        if lon and lat: break
    return lon, lat

def guess_is_geographic(x, y, epsg=None):
    if epsg == 4326: return True
    if epsg and epsg != 4326: return False
    return x.between(-200,200).all() and y.between(-100,100).all()

def utm_zone_from_lon(lon):
    zone = int((lon + 180)//6) + 1
    return 32600 + zone

# ==============================================================
# 3. VOLUME CALCULATION
# ==============================================================
def add_calculated_columns(df):
    df = df.copy()
    df['stem_volume'] = (df['a'] + df['b'] * df['dia_cm'].apply(lambda x: math.log(x)) + df['c'] * df['height_m'].apply(lambda x: math.log(x))).apply(math.exp) / 1000
    df['branch_ratio'] = df['dia_cm'].apply(lambda x: 0.1 if x < 10 else 0.2)
    df['branch_volume'] = df['stem_volume'] * df['branch_ratio']
    df['tree_volume'] = df['stem_volume'] + df['branch_volume']
    df['cm10diaratio'] = (df['a1'] + df['b1'] * df['dia_cm'].apply(lambda x: math.log(x))).apply(math.exp)
    df['cm10topvolume'] = df['stem_volume'] * df['cm10diaratio']
    df['gross_volume'] = df['stem_volume'] - df['cm10topvolume']
    df['net_volume'] = df.apply(lambda row: row['gross_volume'] * 0.9 if row['class'] == 'A' else row['gross_volume'] * 0.8, axis=1)
    df['net_volum_cft'] = df['net_volume'] * 35.3147
    df['firewood_m3'] = df['tree_volume'] - df['net_volume']
    df['firewood_chatta'] = df['firewood_m3'] * 0.105944
    return df

# ==============================================================
# 4. MOTHER TREE ASSIGNMENT
# ==============================================================
def assign_mother_trees(df, grid_spacing=20.0):
    if 'longitude' not in df.columns or 'latitude' not in df.columns:
        raise ValueError("Missing coordinates")

    gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.longitude, df.latitude), crs="EPSG:4326")
    median_lon = df['longitude'].median()
    utm_epsg = utm_zone_from_lon(median_lon)
    gdf_utm = gdf.to_crs(f"EPSG:{utm_epsg}")
    gdf_utm['x_utm'] = gdf_utm.geometry.x
    gdf_utm['y_utm'] = gdf_utm.geometry.y

    gdf_utm['grid_x'] = np.floor(gdf_utm['x_utm'] / grid_spacing).astype(int)
    gdf_utm['grid_y'] = np.floor(gdf_utm['y_utm'] / grid_spacing).astype(int)
    gdf_utm['cell_id'] = gdf_utm['grid_x'].astype(str) + "_" + gdf_utm['grid_y'].astype(str)

    centroids = gdf_utm.groupby('cell_id').agg(cx=('x_utm','mean'), cy=('y_utm','mean')).reset_index()
    gdf_utm = gdf_utm.merge(centroids, on='cell_id')

    def label_mother(g):
        dists = np.sqrt((g['x_utm'] - g['cx'])**2 + (g['y_utm'] - g['cy'])**2)
        mother_idx = dists.idxmin()
        g['status'] = 'felling tree'
        g.loc[mother_idx, 'status'] = 'mother tree'
        return g

    result_gdf = pd.concat([label_mother(g) for _, g in gdf_utm.groupby('cell_id')])
    result_gdf = result_gdf.sort_index()
    result_wgs84 = result_gdf.to_crs("EPSG:4326")

    df_out = df.copy()
    df_out['status'] = result_gdf['status'].values
    df_out['longitude'] = result_wgs84.geometry.x
    df_out['latitude']  = result_wgs84.geometry.y
    df_out['cell_id'] = gdf_utm['cell_id'].values  # Add cell_id
    return df_out

# ==============================================================
# 5. STREAMLIT UI
# ==============================================================
st.set_page_config(page_title="Forest Inventory + Mother Trees", layout="wide")
st.title("Forest Inventory: Volume + Mother Tree Selection")

uploaded_file = st.file_uploader(
    "Upload CSV (must have: `species`, `dia_cm`, `height_m`, `class`, coordinates)",
    type=["csv"]
)

if uploaded_file is not None:
    try:
        df = pd.read_csv(uploaded_file)
        st.success(f"Loaded {len(df):,} trees")
    except Exception as e:
        st.error(f"Error: {e}")
        st.stop()

    # --- Coordinates ---
    st.subheader("1. Coordinate Detection")
    auto_lon, auto_lat = guess_coord_cols(df)
    c1, c2 = st.columns(2)
    with c1:
        lon_col = st.selectbox("X / Easting / Lon", df.columns,
                               index=df.columns.get_loc(auto_lon) if auto_lon in df.columns else 0)
    with c2:
        lat_col = st.selectbox("Y / Northing / Lat", df.columns,
                               index=df.columns.get_loc(auto_lat) if auto_lat in df.columns else 0)

    df[lon_col] = pd.to_numeric(df[lon_col], errors='coerce')
    df[lat_col] = pd.to_numeric(df[lat_col], errors='coerce')
    valid = df[[lon_col, lat_col]].dropna()
    if valid.empty:
        st.error("No valid coordinates.")
        st.stop()

    epsg_in = st.text_input("Source EPSG (optional)", placeholder="32645")
    user_epsg = int(epsg_in) if epsg_in.strip() else None
    is_geo = guess_is_geographic(valid[lon_col], valid[lat_col], user_epsg)

    # --- Determine source EPSG and UTM zone ---
    src_epsg = user_epsg or (4326 if is_geo else utm_zone_from_lon((valid[lon_col].median() - 500000)/111319))

    # Reproject to WGS84
    gdf = gpd.GeoDataFrame(valid, geometry=gpd.points_from_xy(valid[lon_col], valid[lat_col]), crs=f"EPSG:{src_epsg}")
    gdf = gdf.to_crs("EPSG:4326")
    gdf["longitude"] = gdf.geometry.x
    gdf["latitude"]  = gdf.geometry.y

    # --- Compute UTM EPSG for grid (used later) ---
    median_lon = valid[lon_col].median()
    utm_epsg = utm_zone_from_lon(median_lon)  # e.g., 32645

    df_with_coords = df.loc[gdf.index].copy()
    df_with_coords["longitude"] = gdf["longitude"]
    df_with_coords["latitude"]  = gdf["latitude"]

    # --- Species Join ---
    st.subheader("2. Species Parameter Join")
    if "species" not in df_with_coords.columns:
        st.error("`species` column required.")
        st.stop()

    joined_df = df_with_coords.merge(df_ref, left_on="species", right_on="scientific_name", how="left")
    joined_df = joined_df.drop(columns=["scientific_name"], errors="ignore")

    # --- Volume ---
    st.subheader("3. Volume Calculation")
    req = ['dia_cm', 'height_m', 'class']
    if any(c not in joined_df.columns for c in req):
        st.error(f"Missing columns: {[c for c in req if c not in joined_df.columns]}")
        st.stop()

    result_df = joined_df.copy()
    result_df = add_calculated_columns(result_df)
    st.success("Volume calculated")

    # --- Mother Tree Grid ---
    st.subheader("4. Mother Tree Selection (Grid)")
    grid_spacing = st.number_input("Grid size (meters)", 5.0, 200.0, 20.0, 5.0)

    if st.button("Assign Mother Trees"):
        with st.spinner("Processing..."):
            result_df = assign_mother_trees(result_df, grid_spacing)

        st.success(f"{result_df['status'].value_counts().get('mother tree',0)} mother trees identified")

        # --- Results ---
        c1, c2 = st.columns(2)
        with c1:
            st.write("**Mother Trees**")
            st.dataframe(result_df[result_df['status']=='mother tree'][['species','dia_cm','status']].head(10))
        with c2:
            st.write("**Felling Trees**")
            st.dataframe(result_df[result_df['status']=='felling tree'][['species','dia_cm','status']].head(10))

        # --- MAP WITH GRID (WORKING, NO TOKEN NEEDED) ---
        st.subheader("5. Map with Grid (Green = Mother, Red = Felling)")

        # 1. Prepare points
        map_df = result_df[['latitude','longitude','status','cell_id']].copy()
        map_df['lat'] = map_df['latitude']
        map_df['lon'] = map_df['longitude']
        map_df['color'] = map_df['status'].map({
            'mother tree': '#00FF00',
            'felling tree': '#FF0000'
        })

        # 2. Reproject to UTM for grid
        gdf_utm = gpd.GeoDataFrame(
            result_df,
            geometry=gpd.points_from_xy(result_df.longitude, result_df.latitude),
            crs="EPSG:4326"
        ).to_crs(f"EPSG:{utm_epsg}")

        # 3. Create proper GeoJSON grid features
        grid_features = []
        for cell_id in result_df['cell_id'].unique():
            gx, gy = map(int, cell_id.split('_'))
            x0 = gx * grid_spacing
            y0 = gy * grid_spacing
            x1 = x0 + grid_spacing
            y1 = y0 + grid_spacing
            feature = {
                "type": "Feature",
                "geometry": {
                    "type": "Polygon",
                    "coordinates": [[
                        [x0, y0], [x1, y0], [x1, y1], [x0, y1], [x0, y0]
                    ]]
                },
                "properties": {"cell_id": cell_id}
            }
            grid_features.append(feature)

        grid_gdf = gpd.GeoDataFrame.from_features(grid_features, crs=f"EPSG:{utm_epsg}")
        grid_wgs84 = grid_gdf.to_crs("EPSG:4326")

        # 4. Center map
        bounds = grid_wgs84.total_bounds
        center_lat = (bounds[1] + bounds[3]) / 2
        center_lon = (bounds[0] + bounds[2]) / 2

        # 5. Build pydeck with OSM (NO TOKEN!)
        import pydeck as pdk
        import json

        # Grid layer (semi-transparent)
        grid_layer = pdk.Layer(
            "PolygonLayer",
            data=grid_wgs84,
            get_polygon="geometry.coordinates",
            get_fill_color=[200, 200, 200, 50],  # light gray, 20% opacity
            get_line_color=[100, 100, 100],
            get_line_width=2,
            pickable=True,
            auto_highlight=True
        )

        # Points layer
        points_layer = pdk.Layer(
            "ScatterplotLayer",
            data=map_df,
            get_position=["lon", "lat"],
            get_color="color",
            get_radius=8,
            pickable=True,
            tooltip=True
        )

        # View
        view_state = pdk.ViewState(
            latitude=center_lat,
            longitude=center_lon,
            zoom=16,
            pitch=0
        )

        # Deck
        deck = pdk.Deck(
            layers=[grid_layer, points_layer],
            initial_view_state=view_state,
            map_provider="carto",
            map_style="light",
            tooltip={"text": "Status: {status}\nCell: {cell_id}"}
        )

        # Render
        st.pydeck_chart(deck, use_container_width=True)

        # --- Export ---
        st.subheader("6. Download")
        csv = result_df.to_csv(index=False).encode()
        st.download_button("Download Full Result", csv, "forest_mother_trees.csv", "text/csv")
        result_df.to_pickle("result_df_final.pkl")
        st.success("Saved `result_df_final.pkl` – load with `pd.read_pickle('result_df_final.pkl')`")

else:
    st.info("Upload CSV with `species`, `dia_cm`, `height_m`, `class`, and coordinates.")
    st.code("""Example:
species,dia_cm,height_m,class,easting,northing
Shorea robusta,45,25,A,500000,4649776
Shorea robusta,42,24,A,500010,4649780
""")

Overwriting app.py


#for streamlit brower code

In [46]:
from pyngrok import ngrok
import os
import signal

# Authenticate with ngrok using your authtoken
ngrok.set_auth_token("34hVSy1cfuYmYxlminhoFdP7bqf_TcEDNnxWTqzS6VPBPeCH")

# Kill any running ngrok processes
try:
    ngrok.kill()
except:
    pass # Ignore if no ngrok process is running

# Start Streamlit in the background
!streamlit run app.py --server.port=8501 &>/content/logs.txt &

# Create a public URL using ngrok
public_url = ngrok.connect(8501)
print(f"🚀 Your Streamlit app is live at: {public_url}")

🚀 Your Streamlit app is live at: NgrokTunnel: "https://forkier-verna-lathy.ngrok-free.dev" -> "http://localhost:8501"
